In [138]:
import pandas as pd
import numpy as np


def calc_need(dataframe):
    dataframe['today']=dataframe.iloc[:,[2,3,4,5,6,7,8,9]].sum(axis=1).astype(int)
    dataframe['needed']=np.where(dataframe.min_req-dataframe.today<0, 0, dataframe.min_req-dataframe.today)
    return dataframe

def load_data(file, sheet_name):
    dataframe=pd.read_excel(file, sheet_name=sheet_name, index=False)
    dataframe=dataframe.fillna(0)
    calc_need(dataframe)
    #dataframe['today']=(dataframe.sum(axis=1)-dataframe.min_req)-dataframe.priority
    #dataframe['needed']=np.where(dataframe.min_req-dataframe.today<0, 0, dataframe.min_req-dataframe.today)
    total=dataframe.apply(np.sum)
    total.svc='total'
    cols=dataframe.columns.tolist()
    dataframe=dataframe[cols]
    dataframe=dataframe.append(pd.DataFrame(total.values, index=total.keys()).T, ignore_index=True)
    return dataframe

df_curr=load_data('staff.xlsx', 'current')
df_min=load_data('staff.xlsx', 'min')

In [139]:
def get_min(col, idx):
    #for x in df_min.columns:
     #   for y in range(len(df_min)):
    n=df_min[col].values[idx]
    return n

get_min('equip_op', 0)

5.0

In [140]:
df_curr

,svc,priority,cdl,drivers,laborers,admin,cashier,equip_op,supers,emer_staff,min_req,today,needed
0,qrl,1,6,0,12,0,2,0,3,0,12,23,0
1,payroll,1,0,0,0,10,0,0,0,0,9,10,0
2,routine,2,46,0,144,0,0,0,16,0,240,206,34
3,nwts,2,3,0,20,0,1,5,20,0,13,49,0
4,office,3,0,0,0,18,0,0,0,0,2,18,0
5,conv_ctr,3,0,5,10,6,6,5,5,0,14,37,0
6,ds_da,3,9,6,33,0,0,9,2,0,14,59,0
7,mss,3,15,0,0,3,0,0,0,0,2,18,0
8,prop,4,10,5,33,10,0,10,5,0,0,73,0
9,spec,4,6,14,18,0,0,0,0,0,17,38,0


In [149]:
df=df_curr.copy()

def allocate(df):
    '''idx is the row index == service area'''
    '''n is the minimum number of workers required to keep service area running'''
    '''adds 1 to idx when under-staffed'''
    '''starting with lowest priority service, subtract from same column and add to idx row'''
    '''if none available, adds to emergency needs'''
    #report=[]
    for x in df.columns[2:9]:
        for idx in df.index[:-1]:
            n=get_min(x, idx)
            while df[x].values[idx] < n:
                if df[x].values[idx]==n:
                    break
                elif df[x].values[idx+1:len(df)-2].sum()==0:
                    break
                for i in range(len(df)-2, idx+1, -1):
                    if df[x].index[i] > df[x].index[idx]:
                        if df.priority.values[i] > df.priority.values[idx]:
                                if (df[x].values[i] > 0) & (df[x].values[idx] >= n):
                                    pass
                                elif df[x].values[i]==0:
                                    pass
                                elif (df[x].values[i] > 0) & (df[x].values[idx] < n):
                                    count=0
                                    while df[x].values[idx] < n:
                                        if df[x].values[i]==0:
                                            break
                                        elif df[x].values[i] > 0 :
                                            df[x].values[i]-=1
                                            df[x].values[idx]+=1
                                            count+=1
                                    print("Move",count,x.upper(),"from",df.svc.values[i].upper(),"to",df.svc.values[idx].upper())
                                    if df[x].values[i] < get_min(x,i):
                                        print("FLAG:", df.svc.values[i].capitalize(), "does not have minimum", x.capitalize())
                                    #report.append(line)   
            # if df[x].values[idx:len(df)-2].sum() < n:
               #     if df[x].values[idx]+df.emer_staff.values[idx] < n:
               #         df.emer_staff.values[idx]+=(n-(df[x].values[idx:len(df)-2].sum()))
                else:
                    break
    calc_need(df)
    return df #, report

allocate(df)
df

Move 3 CDL from NIGHT to ROUTINE
FLAG: Night does not have minimum Cdl
Move 6 CDL from SPEC to ROUTINE
FLAG: Spec does not have minimum Cdl
Move 10 CDL from PROP to ROUTINE
Move 7 CDL from MSS to ROUTINE
Move 6 DRIVERS from SPEC to NWTS
Move 2 LABORERS from NIGHT to ROUTINE
FLAG: Night does not have minimum Laborers
Move 2 LABORERS from MARINE to ROUTINE
Move 1 ADMIN from NIGHT to ROUTINE
Move 3 ADMIN from MARINE to ROUTINE
Move 1 ADMIN from MARINE to DS_DA
Move 4 EQUIP_OP from MARINE to QRL
FLAG: Marine does not have minimum Equip_op
Move 1 EQUIP_OP from PROP to QRL
Move 1 SUPERS from NIGHT to PAYROLL
FLAG: Night does not have minimum Supers
Move 1 SUPERS from MARINE to PAYROLL


,svc,priority,cdl,drivers,laborers,admin,cashier,equip_op,supers,emer_staff,min_req,today,needed
0,qrl,1,6,0,12,0,2,5,3,0,12,28,0
1,payroll,1,0,0,0,10,0,0,2,0,9,12,0
2,routine,2,72,0,148,4,0,0,16,0,240,240,0
3,nwts,2,3,6,20,0,1,5,20,0,13,55,0
4,office,3,0,0,0,18,0,0,0,0,2,18,0
5,conv_ctr,3,0,5,10,6,6,5,5,0,14,37,0
6,ds_da,3,9,6,33,1,0,9,2,0,14,60,0
7,mss,3,8,0,0,3,0,0,0,0,2,11,0
8,prop,4,0,5,33,10,0,9,5,0,0,62,0
9,spec,4,0,8,18,0,0,0,0,0,17,26,0
